In [ ]:
from ipynb.fs.full.procc import *
import matplotlib.cm as cm
from matplotlib import colors
import csv
from scipy.interpolate import interp2d
import sys
from IPython.display import clear_output

In [ ]:
def procFiles(for_direc, file_type, start_year, end_year):
    
    file_dict = {}
    # dates = ['0346','0952']
    
    #creates a dictionary with the file name as the key and the path of the file as the value
    for root, dirs, filenames in os.walk(for_direc):
        for file in filenames:
            if file.endswith(".nc") and file_type in file and file[3:7] >= start_year and file[3:7] < end_year and 'aijh1' not in file : #** (aijlh1) change format depending on scaleacc processing, may have to add enddate so doesn't go to 0101-1953, also aij
                file_to_add = os.path.join(root + '/', file) 
                file_dict[file] = file_to_add
                       
    return file_dict

In [ ]:
def openCSV(fn):
    array = []

    with open(fn, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            remake_row = []
            for num in row:
                remake_row.append(float(num))
            array.append(remake_row)

    return np.array(array)

In [ ]:
# Interpolation functions:
def interpNans(x,y,z):
    
    nan_map = np.zeros_like( z )
    nan_map[ np.isnan(z) ] = 1

    filled_z = z.copy()
    filled_z[ np.isnan(z) ] = 0

    f = interp2d(x, y, filled_z, kind='linear')
    f_nan = interp2d(x, y, nan_map, kind='linear')     

    # Interpolation on new points:
    xnew = np.linspace(np.min(x), np.max(x), len(x)*2-1)
    ynew = np.linspace(np.min(y), np.max(y), len(y)*2-1)

    z_new = f(xnew, ynew)
    nan_new = f_nan( xnew, ynew )
    z_new[ nan_new>0.5 ] = np.nan
    
    return xnew,ynew,z_new

In [ ]:
def remNan(array):
    
    replace_val = np.nanmean(array)
    
    for i in range(0,len(array)):
        for j in range(0,len(array[i])):
            if np.isnan(array[i,j]):
                array[i,j] = replace_val
    
    return array
    

In [ ]:
def graphContour(x,y,z,savename): #levels=np.linspace(np.nanmin(z),np.nanmax(z),8)):
    plt.clf()

    plt.figure(figsize=(12, 6), dpi=100)
    plt.rcParams.update({'font.size': 14})

    plt.xlim([-180,180])
    plt.ylim([-90,90])

    plt.yticks([-90,-60,-30,0,30,60,90])
    plt.xticks([-180,-150,-120,-90,-60,-30,0,30,60,90,120,150,180])

    plt.xlabel('Longitude (°)')
    plt.ylabel('Latitude (°)')
    
    #levels=np.linspace(np.nanmin(z),np.nanmax(z),8)
    
    plt.contourf(x,y,z, levels=np.arange(-1.2,1.3,0.2))
    plt.colorbar(label='Wind Velocity (m/s)')
    
    plt.savefig('/Users/maxcollins/Desktop/organized images/Contours/longitude latitude/'+run+'_'+savename+'Contour_latlon.png', bbox_inches='tight')
    #plt.savefig('/home/maxqc/images/Contours/'+run+'_'+savename+'Contour_latlon.png', bbox_inches='tight')
    plt.show()


In [ ]:
# direc = str(sys.argv[1])
# file_type = str(sys.argv[2])            # aij
# start_year = str(sys.argv[3])
# end_year = str(sys.argv[4])
# run = str(sys.argv[5]) # year after the last wanted

In [ ]:
direc = "/Volumes/ExHDD/TitanWTZ0_10yrDaily"
file_type = "aij"           # aij
start_year = "1960"
end_year = "1971"
run = "Z0" # year after the last wanted

In [ ]:
procDirec = procFiles(direc, file_type, start_year, end_year)

In [ ]:
lats = np.arange(-90., 91., 4)
lons = np.arange(-177.5, 177.6, 5)

u_grid = np.zeros([len(lats),len(lons)])
v_grid = np.zeros([len(lats),len(lons)])

count_v_grid = np.zeros([len(lats),len(lons)])
count_u_grid = np.zeros([len(lats),len(lons)])

In [ ]:
count=0
total = len(lats)*len(lons)*len(procDirec.keys())

for key in procDirec.keys(): # Year 0, Year 1, ...
        
        with xr.open_dataset(procDirec[key]) as ds:

            for i, lat in enumerate(lats):

                for j, lon in enumerate(lons):

                    u = ds.usurf.sel(lat=lat,lon=lon)
                    v = ds.vsurf.sel(lat=lat,lon=lon)

                    if not np.isnan(u):
                        u_grid[i,j] = u_grid[i,j] + u   
                        count_u_grid[i,j] += 1 
                    
                    if not np.isnan(v):
                        v_grid[i,j] = v_grid[i,j] + v
                        count_v_grid[i,j] += 1
                    
                    count+=1
                    
                    perc=float(count/total)*100
                    
                    text = "Percent completed: {:.2f}% ".format(perc)
                    
                    print(text)
                    
                    clear_output(wait=True)
            
for i in range(0,len(lats)):
    
    for j in range(0,len(lons)):
        
        if float(u_grid[i,j]):
            u_grid[i,j] = 1.0*u_grid[i,j]/count_u_grid[i,j]
        if float(v_grid[i,j]):
            v_grid[i,j] = 1.0*v_grid[i,j]/count_v_grid[i,j]

clear_output(wait=True)

np.savetxt("/Users/maxcollins/Documents/Code/python/data/"+run+"AIJv_grid.csv", v_grid, delimiter=",")
np.savetxt("/Users/maxcollins/Documents/Code/python/data/"+run+"AIJu_grid.csv", u_grid, delimiter=",")

In [ ]:
# u = openCSV("/home/maxqc/Python/data/"+run+"AIJu_grid.csv")
# v = openCSV("/home/maxqc/Python/data/"+run+"AIJv_grid.csv")

u = openCSV("/Users/maxcollins/Documents/Code/python/data/"+run+"/"+run+"AIJu_grid.csv")
v = openCSV("/Users/maxcollins/Documents/Code/python/data/"+run+"/"+run+"AIJv_grid.csv")

In [ ]:
x = lons
y = lats
z = remNan(u)

savename = 'U'
graphContour(x,y,z,savename)

In [ ]:
x = lons
y = lats
z = remNan(v)

savename = 'V'
graphContour(x,y,z,savename)

In [ ]:
x, y, z = interpNans(lons,lats,u)


savename = 'U_interp'
graphContour(x,y,z,savename)

In [ ]:
x, y, z = interpNans(lons,lats,v)
#x, y, z = interpNans(x,y,z)
savename = 'V_interp'
graphContour(x,y,z,savename)